In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import avg, max

# Create a SparkSession

In [4]:
spark = SparkSession.builder \
    .appName("ExploratoryDataAnalysis") \
    .getOrCreate()
df = spark.read.csv('E:/level 2/23_24_premier_league_0408_Squad Advanced Goalkeeping.csv', header=True, escape='"')

# display the data

In [5]:
df.show()

+---------------+--------------+----------------------+--------+---------+--------+--------+--------+-------------+-----------------+----------------+------------+------------+------------+-------------+---------------+----------+--------------+-------------+--------------+------------------+-----------------+-----------+-----------+------------+------------+---------------+---------------+
|          Squad|Num_of_players|Mins_played_diveded_90|Goals_GA|Goals_PKA|Goals_FK|Goals_CK|Goals_OG|Expected_PSxG|Expected_PSxG/SoT|Expected_PSxG+/-|Expected_/90|Launched_Cmp|Launched_Att|Launched_Cmp%|Passes_Att (GK)|Passes_Thr|Passes_Launch%|Passes_AvgLen|Goal Kicks_Att|Goal Kicks_Launch%|Goal Kicks_AvgLen|Crosses_Opp|Crosses_Stp|Crosses_Stp%|Sweeper_#OPA|Sweeper_#OPA/90|Sweeper_AvgDist|
+---------------+--------------+----------------------+--------+---------+--------+--------+--------+-------------+-----------------+----------------+------------+------------+------------+-------------+---------

# show the Schema

In [4]:
df.printSchema()

root
 |-- Squad: string (nullable = true)
 |-- Num_of_players: string (nullable = true)
 |-- Mins_played_diveded_90: string (nullable = true)
 |-- Goals_GA: string (nullable = true)
 |-- Goals_PKA: string (nullable = true)
 |-- Goals_FK: string (nullable = true)
 |-- Goals_CK: string (nullable = true)
 |-- Goals_OG: string (nullable = true)
 |-- Expected_PSxG: string (nullable = true)
 |-- Expected_PSxG/SoT: string (nullable = true)
 |-- Expected_PSxG+/-: string (nullable = true)
 |-- Expected_/90: string (nullable = true)
 |-- Launched_Cmp: string (nullable = true)
 |-- Launched_Att: string (nullable = true)
 |-- Launched_Cmp%: string (nullable = true)
 |-- Passes_Att (GK): string (nullable = true)
 |-- Passes_Thr: string (nullable = true)
 |-- Passes_Launch%: string (nullable = true)
 |-- Passes_AvgLen: string (nullable = true)
 |-- Goal Kicks_Att: string (nullable = true)
 |-- Goal Kicks_Launch%: string (nullable = true)
 |-- Goal Kicks_AvgLen: string (nullable = true)
 |-- Crosses_

In [5]:
df.count()

20

# Question 1: Which team has the highest average minutes played per game?

In [6]:
avg_minutes_per_game = df.groupBy("Squad").agg(avg("Mins_played_diveded_90").alias("Avg_Minutes_Played"))
max_avg_minutes = avg_minutes_per_game.agg({"Avg_Minutes_Played": "max"}).collect()[0][0]
team_with_max_avg_minutes = avg_minutes_per_game.filter(avg_minutes_per_game["Avg_Minutes_Played"] == max_avg_minutes).select("Squad").collect()[0][0]
print("Team with the highest average minutes played per game:", team_with_max_avg_minutes, "with an average of", max_avg_minutes, "minutes per game")

Team with the highest average minutes played per game: Brentford with an average of 32.0 minutes per game


# Question 2: What is the distribution of goals (GA) across teams?

In [7]:

goals_distribution = df.groupBy("Squad").agg(avg("Goals_GA").alias("Avg_Goals"))
print("Distribution of goals across teams:")
goals_distribution.show()

Distribution of goals across teams:
+---------------+---------+
|          Squad|Avg_Goals|
+---------------+---------+
|      Tottenham|     45.0|
|       Brighton|     49.0|
|  Sheffield Utd|     82.0|
|        Arsenal|     24.0|
|      Brentford|     58.0|
|  Newcastle Utd|     52.0|
| Crystal Palace|     54.0|
|        Burnley|     67.0|
|    Aston Villa|     49.0|
|Manchester City|     31.0|
|    Bournemouth|     55.0|
| Manchester Utd|     46.0|
|         Fulham|     51.0|
|       West Ham|     56.0|
|Nott'ham Forest|     56.0|
|         Wolves|     49.0|
|      Liverpool|     30.0|
|        Chelsea|     52.0|
|     Luton Town|     65.0|
|        Everton|     42.0|
+---------------+---------+



# Question 3: Which team has the highest pass completion percentage for launched passes?

In [8]:
pass_completion_percentage = df.withColumn("Pass_Completion_Percentage", (df["Launched_Cmp"] / df["Launched_Att"]) * 100)
max_pass_completion = pass_completion_percentage.agg({"Pass_Completion_Percentage": "max"}).collect()[0][0]
team_with_max_pass_completion = pass_completion_percentage.filter(pass_completion_percentage["Pass_Completion_Percentage"] == max_pass_completion).select("Squad").collect()[0][0]
print("Team with the highest pass completion percentage for launched passes:", team_with_max_pass_completion, "with a pass completion percentage of", max_pass_completion)

Team with the highest pass completion percentage for launched passes: Manchester City with a pass completion percentage of 46.86346863468634


# Question 4: Which team has the highest percentage of crosses stopped defensively?

In [10]:

crosses_stopped_percentage = df.withColumn("Crosses_Stp_Percentage", (df["Crosses_Stp"] / df["Crosses_Opp"]) * 100)
max_crosses_stopped_percentage = crosses_stopped_percentage.agg({"Crosses_Stp_Percentage": "max"}).collect()[0][0]
team_with_max_crosses_stopped = crosses_stopped_percentage.filter(crosses_stopped_percentage["Crosses_Stp_Percentage"] == max_crosses_stopped_percentage).select("Squad").collect()[0][0]
print("Team with the highest percentage of crosses stopped defensively:", team_with_max_crosses_stopped, "with a defensive crosses stopped percentage of", max_crosses_stopped_percentage)

Team with the highest percentage of crosses stopped defensively: Arsenal with a defensive crosses stopped percentage of 13.651877133105803


# Question 5: Which team has the highest number of sweeper actions per 90 minutes?

In [13]:

sweeper_actions = df.groupBy("Squad").agg(avg("Sweeper_#OPA/90").alias("Avg_Sweeper_Actions_per_90"))
max_sweeper_actions = sweeper_actions.agg({"Avg_Sweeper_Actions_per_90": "max"}).collect()[0][0]
team_with_max_sweeper_actions = sweeper_actions.filter(sweeper_actions["Avg_Sweeper_Actions_per_90"] == max_sweeper_actions).select("Squad").collect()[0][0]
print("Team with the highest number of sweeper actions per 90 minutes:", team_with_max_sweeper_actions, "with an average of", max_sweeper_actions, "sweeper actions per 90 minutes")

Team with the highest number of sweeper actions per 90 minutes: Aston Villa with an average of 2.03 sweeper actions per 90 minutes


# Stop the SparkSession

In [16]:

spark.stop()